In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans


In [2]:
p1_data = pd.read_csv('data/P1.txt', delimiter=' ')

In [4]:
DEPOT_LOCATION = (-14, 9)

# Our functions import

In [ ]:
from generate_population_scripts import generate_population, evaluate_distance_fitness, get_evalution_count, set_evaluation_count
from combination import cut_and_crossfil, mutation_inverse, mutation_scramble
from selection import roulette_wheel, binary_tournament

# PROBLEM No. 1

In [11]:
pop_arr = generate_population(70, DEPOT_LOCATION, p1_data)

In [26]:
def algorithm_run(pop_count, dataset, depot_location_dict, depot_symbol, max_capacity, vehicle_count, selection_method, FITNESS_FUNCTION, MUTATION_METHOD, RECOMBINATION_METHOD, p_m=0.1, p_c =0.9 ,max_evaluations = 5000):
    """
    depot_symbol is just one string
    """
    ## reset the evaluation counts
    set_evaluation_count(0)

    population, fitness_pop = generate_population(max_capacity, depot_location_dict[depot_symbol], dataset, depot_symbol, pop_count, vehicle_count)
    generation_count = 0

    best_chromosome = None
    best_chromsome_fitness = None

    while get_evalution_count() < max_evaluations:
        ## create pair of the parents
        parent_pairs = []
        for _ in range(pop_count):
            pair = selection_method(population, fitness_pop)
            parent_pairs.append(pair)

        
        offsprings = []
        fitness_offsprings = []
        for parents in parent_pairs:
            recombination_p = np.random.random()

            ## the offspring for this iteration
            iteration_offspring = []
            
            ######## Recombination ########
            if recombination_p < p_c:
                offspring1, offspring2 =  RECOMBINATION_METHOD(parents[0], parents[1], dataset, max_capacity, [depot_symbol])

                iteration_offspring = [offspring1, offspring2]

            
            ######## Mutation ########
            mutation_p = np.random.random()

            ## if cross over has happend
            if len(iteration_offspring) != 0 and mutation_p < p_m:
                offspring1 = MUTATION_METHOD(iteration_offspring[0], max_capacity, dataset, [depot_symbol])
                offspring2 = MUTATION_METHOD(iteration_offspring[1], max_capacity, dataset, [depot_symbol])

                iteration_offspring = [offspring1, offspring2]
            ## if cross over had not happened
            elif mutation_p < p_m:
                offspring1 = MUTATION_METHOD(parents[0], max_capacity, dataset, [depot_symbol])
                offspring2 = MUTATION_METHOD(parents[1], max_capacity, dataset, [depot_symbol])

                iteration_offspring = [offspring1, offspring2]
            ## if both cross over and mutation has not happened!
            else:
                iteration_offspring = [parents[0], parents[1]]
            
            ## finally append the genarated offsprings to offspring array 
            offsprings.append(iteration_offspring[0])
            offsprings.append(iteration_offspring[1])

            fitness_offsprings.append(FITNESS_FUNCTION(iteration_offspring[0], depot_location_dict[depot_symbol], dataset))
            fitness_offsprings.append(FITNESS_FUNCTION(iteration_offspring[1], depot_location_dict[depot_symbol], dataset))

        ## replace the old population with the new ones

        ## the whole generation: parents + offsprings
        generation_population = population.copy()
        generation_population.extend(offsprings)

        ## whole generation fitness: parents fitness + offsprings fitness
        generation_fitness = fitness_pop.copy()
        generation_fitness.extend(fitness_offsprings)

        ## the sorted generation
        generation_population_sorted = np.array(generation_population)[np.argsort(generation_fitness)]
        generation_fitness_sorted = np.sort(generation_fitness)

        ## Step 10
        ## extract the best of the new generation

        best_of_generation_population = generation_population_sorted[:pop_count]
        best_of_generation_fitness = generation_fitness_sorted[:pop_count]

        best_chromosome = generation_population_sorted[0]
        best_chromsome_fitness = generation_fitness_sorted[0]

        ## save them into the original population arrays
        population = best_of_generation_population.tolist()
        fitness_pop = best_of_generation_fitness.tolist()

        ## increase the generation value
        generation_count += 1
    
    return best_chromosome, best_chromsome_fitness

In [35]:
best_chromsome, best_chromsome_fitness = algorithm_run(pop_count = 50, 
                dataset=p1_data,
                depot_location_dict= {'(1)': DEPOT_LOCATION},
                depot_symbol='(1)',
                max_capacity=70, 
                vehicle_count=6, 
                selection_method=binary_tournament,
                FITNESS_FUNCTION=evaluate_distance_fitness,
                MUTATION_METHOD=mutation_scramble,
                RECOMBINATION_METHOD=cut_and_crossfil,
                max_evaluations=5000)

best_chromsome, best_chromsome_fitness

('(1)174116170163140(1)|(1)176144149(1)167131161123(1)124128118142110(1)195127164162(1)196146180182141(1)148175111171151139(1)|(1)197168115169184(1)154192109166112119190(1)|(1)150133147106137(1)193126200(1)102187173138135120(1)189125113172191(1)|(1)134143156159117(1)101177179(1)136121130157158103178(1)153152181183107199104(1)108185132160114194198(1)105188155122145186(1)|(1)165129(1)',
 6088)